In [9]:
import numpy as np
import keras.backend as K
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
K.set_image_data_format('channels_first') 
seed = 7 
np.random.seed(seed)


In [59]:
def get_data_mnist():
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32') 
    X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32') 
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    num_classes = y_test.shape[1]
    return (X_train, y_train), (X_test, y_test), num_classes

In [61]:
def train_and_evaluate(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=1)
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("Model score : %.2f%%" % (scores[1] * 100))
    print("Model error rate : %.2f%%" % (100 - scores[1] * 100))

In [65]:
def print_model_error_rate(model, X_test, y_test):
    # Final evaluation of the model
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("Model score : %.2f%%" % (scores[1]*100))
    print("Model error rate : %.2f%%" % (100-scores[1]*100))

In [71]:
def small_model(num_classes):
    model = Sequential()
    model.add(Conv2D(64, (3, 3), input_shape=(28, 28, 1), activation='relu', padding='same'))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


num_classes = 10
(X_train, y_train), (X_test, y_test), num_classes = get_data_mnist()
small_cnn = small_model(10)
train_and_evaluate(small_cnn, X_train, y_train, X_test, y_test)

Epoch 1/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.7098 - loss: 3.1578 - val_accuracy: 0.9173 - val_loss: 0.3000
Epoch 2/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9342 - loss: 0.2338 - val_accuracy: 0.9447 - val_loss: 0.1871
Epoch 3/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9555 - loss: 0.1551 - val_accuracy: 0.9525 - val_loss: 0.1649
Epoch 4/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9678 - loss: 0.1076 - val_accuracy: 0.9607 - val_loss: 0.1345
Epoch 5/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9753 - loss: 0.0818 - val_accuracy: 0.9659 - val_loss: 0.1206
Epoch 6/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9783 - loss: 0.0701 - val_accuracy: 0.9671 - val_loss: 0.1207
Epoch 7/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9814 - loss: 0.0606 - val_accuracy: 0.9664 - val_loss: 0.1192
Epoch 8/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9818 - loss: 0.0549 - val_accu

In [70]:
small_cnn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 24, 24, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 18432)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │       184,330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 610,304 (2.33 MB)

 Trainable params: 203,434 (794.66 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 406,870 (1.55 MB)

In [72]:
print_model_error_rate(small_cnn, X_test, y_test)

Model score : 97.13%
Model error rate : 2.87%


In [5]:
def save_keras_model(model, filename):
    model_json = model.to_json()
    with open(filename + ".json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights(filename + ".h5")

In [6]:
def load_keras_model(filename):
    json_file = open(filename + ".json", 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights(filename + ".h5")
    return loaded_model

In [81]:
save_keras_model(small_cnn, "small_cnn.weights")

In [22]:
load_keras_model( "small_cnn.weights")

<Sequential name=sequential, built=True>

In [74]:
def get_data_mnist():
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32') / 255
    X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32') / 255
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    num_classes = y_test.shape[1]
    return (X_train, y_train), (X_test, y_test), num_classes

In [75]:
def medium_cnn():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=(28, 28, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

(X_train, y_train), (X_test, y_test), num_classes = get_data_mnist()
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32') / 255
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32') / 255
medium_cnn = medium_cnn()
train_and_evaluate(medium_cnn, X_train, y_train, X_test, y_test)

Epoch 1/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.2729 - loss: 2.1190 - val_accuracy: 0.7505 - val_loss: 0.9336
Epoch 2/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7509 - loss: 0.8432 - val_accuracy: 0.8404 - val_loss: 0.5461
Epoch 3/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8196 - loss: 0.5847 - val_accuracy: 0.8702 - val_loss: 0.4442
Epoch 4/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8472 - loss: 0.4924 - val_accuracy: 0.8911 - val_loss: 0.3807
Epoch 5/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8672 - loss: 0.4347 - val_accuracy: 0.9015 - val_loss: 0.3415
Epoch 6/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8825 - loss: 0.3929 - val_accuracy: 0.9114 - val_loss: 0.3079
Epoch 7/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8936 - loss: 0.3594 - val_accuracy: 0.9191 - val_loss: 0.2828
Epoch 8/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8985 - loss: 0.3396 - val_accuracy: 0.

In [80]:
print_model_error_rate(medium_cnn, X_test, y_test)
save_keras_model(medium_cnn, "medium_cnn.weights")

Model score : 89.13%
Model error rate : 10.87%


In [77]:
def large_cnn():
    model = Sequential()
    model.add(Conv2D(30, (5, 5), padding='same', input_shape=(28, 28, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

(X_train, y_train), (X_test, y_test), num_classes = get_data_mnist()
large_cnn = large_cnn()
train_and_evaluate(large_cnn, X_train, y_train, X_test, y_test)


Epoch 1/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.7353 - loss: 0.8717 - val_accuracy: 0.9486 - val_loss: 0.1747
Epoch 2/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.9414 - loss: 0.1957 - val_accuracy: 0.9624 - val_loss: 0.1195
Epoch 3/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9586 - loss: 0.1327 - val_accuracy: 0.9706 - val_loss: 0.0934
Epoch 4/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9656 - loss: 0.1139 - val_accuracy: 0.9722 - val_loss: 0.0825
Epoch 5/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9726 - loss: 0.0920 - val_accuracy: 0.9761 - val_loss: 0.0771
Epoch 6/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9754 - loss: 0.0815 - val_accuracy: 0.9794 - val_loss: 0.0654
Epoch 7/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9785 - loss: 0.0716 - val_accuracy: 0.9805 - val_loss: 0.0597
Epoch 8/10
300/300 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9792 - loss: 0.0658 - val_accu

In [79]:
print_model_error_rate(large_cnn, X_test, y_test)
save_keras_model(large_cnn, "large_cnn.weights")

Model score : 98.24%
Model error rate : 1.76%
